# Module Imports

In [1]:
## IMPORT MODULES
import pandas as pd
import numpy as np
from sklearn import linear_model


In [2]:
## IMPORT PLOTTING MODULES
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import matplotlib as mpl
from matplotlib import cm
%matplotlib widget

# Dataframe

In [3]:
## FULL DATAFRAME
df_raw = pd.read_csv("WL_2-Policy_Summary.csv")

## Drop extraneous data points
df_raw = df_raw.drop(df_raw[
    (df_raw['landing_rate']<=0.1) & 
    (df_raw['vz_d']>=2.75)
    ].index)

df_raw = df_raw.drop(df_raw[
    (df_raw['landing_rate']<=0.1) & 
    (df_raw['vx_d']>=1.0)
    ].index)

df_raw_avg = df_raw.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()


# df_temp = df.query(f"landing_rate>={0.2}")
# df_temp_avg = df_raw.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()


df = df_raw
df['My_d'] = df['My_d'].apply(lambda x: np.abs(x)) # Convert My to magnitude
# df['My_d'] = df['My_d'].apply(lambda x: 7.7 if x>7.7 else x) # Cap My_d to max moment
# df = df.drop(df[(df['vz_d']<= 2.25) & (df['vx_d']<= 0.75)].index) # Drop corner with no successful landings
df = df.dropna()

## AVERAGED DATAFRAME
df_avg = df.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()

In [4]:
df

,vz_d,vx_d,trial_num,landing_rate,RREV_threshold,G1,RREV_sig,G1_sig,RREV_trigger,OF_y,My_d,impact_eul,impact_tdelta,alpha_mu,alpha_sigma,mu_ini,sigma_ini
0,1.5,0.00,0,0.272727,4.59,4.95,0.00,0.05,5.021833,0.000000,4.937833,-97.088206,0.199667,[0. 0.],[0. 0.],[2.81 3.88],[1.5 1.5]
1,1.5,0.00,1,0.250000,5.15,5.66,0.01,0.00,5.418167,0.000000,5.661000,-91.943326,0.173333,[0. 0.],[0. 0.],[1.22 4.45],[1.5 1.5]
3,1.5,0.00,3,0.291667,4.46,7.38,0.01,0.59,4.897286,0.000000,7.344286,-118.566499,0.168571,[0. 0.],[0. 0.],[5.54 6.08],[1.5 1.5]
4,1.5,0.00,4,0.416667,4.74,5.31,0.01,0.01,5.268700,-0.000100,5.315900,-95.375665,0.182400,[0. 0.],[0. 0.],[1.76 6.35],[1.5 1.5]
5,1.5,0.00,5,0.600000,4.79,3.22,0.00,0.87,5.411333,-0.000083,3.454833,-86.938026,0.207500,[0. 0.],[0. 0.],[6.31 6.04],[1.5 1.5]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,4.0,2.75,1,0.916667,4.78,4.77,0.02,0.00,5.082955,-3.318864,4.774864,-118.326346,0.208636,[0. 0.],[0. 0.],[5.19 2.78],[1.5 1.5]
766,4.0,2.75,2,1.000000,4.62,4.14,0.04,0.00,4.938167,-3.220208,4.139125,-126.525807,0.226250,[0. 0.],[0. 0.],[6.01 4.09],[1.5 1.5]
767,4.0,2.75,3,0.833333,4.72,4.47,0.02,0.02,5.045650,-3.288900,4.473750,-118.412714,0.220300,[0. 0.],[0. 0.],[4.16 4.52],[1.5 1.5]
768,4.0,2.75,4,0.916667,4.24,2.22,0.01,0.04,4.537500,-2.959773,2.249909,-99.818951,0.263455,[0. 0.],[0. 0.],[6.47 2.14],[1.5 1.5]


# Landing Rate Data

In [5]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
ax.scatter(df['vx_d'],df['vz_d'],df['landing_rate'])

ax.set_zlim(0,1)

ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Landing Rate')
ax.set_title('Landing Rate (Raw Data)')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
## AVG LANDING RATE SURFACE
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0,vmax=1)
pcm = ax.plot_trisurf(df_raw_avg['vx_d'],df_raw_avg['vz_d'],df_raw_avg['landing_rate','mean'],cmap = cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing %")
df
ax.set_zlim(0,1)

ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Landing Rate')
ax.set_title('Avg Landing Rate for Final 3 Episodes')


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
## AVG LANDING RATE SURFACE
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

from scipy.interpolate import griddata


X = df_raw_avg['vx_d']
Y = df_raw_avg['vz_d']
Z = df_raw_avg['landing_rate','mean']

xi = np.linspace(X.min(),X.max(),(len(Z)//3))
yi = np.linspace(Y.min(),Y.max(),(len(Z)//3))
zi = griddata((X, Y), Z, (xi[None,:], yi[:,None]), method='linear')
xig, yig = np.meshgrid(xi, yi)

cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0,vmax=1)


surf = ax.plot_surface(xig, yig, zi,cmap=cmap,norm=norm)
# surf = ax.contour(xig, yig, zi,cmap=cmap,norm=norm)

fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="Landing %")
ax.set_zlim(0,1)

ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Landing Rate')
ax.set_title('Avg Landing Rate for Final 3 Episodes')


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# RREV vs IC

In [8]:
## RREV Threshold vs IC
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

ax.scatter(df['vx_d'],df['vz_d'],df['RREV_threshold'])


ax.set_xlim(0,2.75)
ax.set_ylim(1.0,4.5)


ax.set_xlabel('Vx_d')
ax.set_ylabel('Vz_d')
ax.set_zlabel('RREV_threshold')
ax.set_title('RREV_thr vs IC - (Raw Data)')


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:

## Avg RREV_thr vs IC
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=4,vmax=5)
pcm = ax.plot_trisurf(df_avg['vx_d'],df_avg['vz_d'],df_avg['RREV_threshold','mean'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="RREV_threshold")

ax.set_xlim(0,2.75)
ax.set_ylim(1.0,4.5)
ax.set_zlim(3,6)

ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('RREV_threshold')

ax.set_title('Avg RREV_thr vs IC')
plt.show()







Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Rotation Time Data

In [10]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

df_temp = df.query(f"landing_rate>={0.1}")

cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=4,vmax=5)
pcm = ax.scatter(df_temp['vx_d'],df_temp['vz_d'],df_temp['impact_tdelta'])
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label="RREV_threshold")


ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Delta_t [s]')

ax.set_title('Time Rotating vs IC (Raw Data)')

ax.set_zlim(0,0.5)




plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

df_temp = df.query(f"landing_rate>={0.1}")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=4)

pcm = ax.scatter(df_temp['RREV_trigger'],df_temp['landing_rate'],df_temp['impact_tdelta'],c=df_temp['vz_d'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Vz_d')


ax.set_xlabel('RREV_trigger')
ax.set_ylabel('Landing_rate')
ax.set_zlabel('Delta_t [s]')

ax.set_title('Time Rotating vs RREV (Raw Data)')

ax.set_zlim(0,0.5)




plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# My_d vs IC Data

In [12]:
## Define Dataframe

df_temp = df.query(f"landing_rate>={0.1}")


fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

pcm = ax.scatter(df['vx_d'],df['vz_d'],df['My_d'])



ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('My_d [N*mm]')

ax.set_title('My_d vs IC (Raw Data)')

## I have more data for vz = (1.5,2.75) than vz = (3.00,4.0) so My_d differences are likely not a trend, just sparse data




plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Impact Angle Data

In [13]:
df_temp = df.query(f"impact_eul<={-60}")
# df_temp_avg = df_raw.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()



## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
pcm = ax.scatter(df_temp['vx_d'],df_temp['vz_d'],-df_temp['impact_eul'],c=df_temp['landing_rate'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing %')

ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Impact Angle')


ax.set_zlim(60,180)
ax.set_zticks([60,90,120,150,180])

## Failed landings in corner
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
## CLEAN DATAFRAME
df_temp = df.query(f"impact_eul<={-60}")
df_temp_avg = df_temp.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()


## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0,vmax=180)
ax.plot_trisurf(df_temp_avg['vx_d'],df_temp_avg['vz_d'],-df_temp_avg['impact_eul','mean'],cmap=cmap)


ax.set_xlim(0,2.75)
ax.set_ylim(1.0,4.0)
ax.set_zlim(60,180)
ax.set_zticks([60,90,120,150,180])


ax.set_xlabel('vx_d')
ax.set_ylabel('vz_d')
ax.set_zlabel('Impact Angle')

ax.set_title('Avg Impact Angle vs IC')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Moment vs RREV

In [15]:
## CLEAN DATAFRAME
df_temp = df.query(f"My_d<={7.7}")
df_temp_avg = df_temp.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()


## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111)


cmap = mpl.cm.Greys
norm = mpl.colors.Normalize(vmin=0,vmax=1)

ax.scatter(df_temp['RREV_threshold'],df_temp['My_d'],c=df_temp['landing_rate'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing %')

ax.grid()
ax.set_xlim(2,7)
ax.set_ylim(0,10)
ax.hlines(7.77,2,7)
ax.text(2.05,8.0,'Motors Maxed Out')

ax.set_xlabel('RREV_thr')
ax.set_ylabel('My_d [N*mm]')
ax.set_title('Policy: My_d vs RREV_thr')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
## CLEAN DATAFRAME
df_temp = df.query(f"My_d<={7.7}")
df_temp_avg = df_temp.groupby(['vz_d','vx_d']).agg([np.mean,np.std]).reset_index()

## AVG LANDING RATE PLOT
fig = plt.figure()
ax = fig.add_subplot(111)


cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0,vmax=3)

ax.scatter(df_temp['RREV_threshold'],df_temp['My_d'],c=df_temp['vz_d'],cmap=cmap)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Vz_d')

ax.grid()
ax.set_xlim(2,7)
ax.set_ylim(0,10)
ax.hlines(7.77,2,7)
ax.text(2.05,8.0,'Motors Maxed Out')

ax.set_xlabel('RREV_thr')
ax.set_ylabel('My_d [N*mm]')
ax.set_title('Policy: My_d vs RREV_thr')
## As Vz increases RREV_trigger decreases to preserve time to contact. It also finds a host of My_d that work and they vary because of the robustness of the legs and the fact that landing success isn't deterministic.
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# POLICY FITTING

## RAW POLICY RELATION

In [17]:
## CLEAN DATAFRAME
df_temp = df.query(f"My_d<={7.7}") # Remove extraneous moments higher than max

## PLOT DATA
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)

ax.scatter(df_temp['RREV_threshold'],df_temp['OF_y'],df_temp['My_d'],c=df_temp['landing_rate'],cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing Rate')

ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)

ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')

ax.set_title('Raw Policy Relation')


plt.show()

# Note: the planar look to the data

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
## CLEAN DATAFRAME
df_temp = df.query(f"My_d<={7.7}") # Remove extraneous moments higher than max
df_temp = df_temp.query(f"RREV_threshold>{3.5}") # Remove outlier data
df_temp = df_temp.query(f"landing_rate>={0.8}")

## PLOT DATA
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)

ax.scatter(df_temp['RREV_threshold'],df_temp['OF_y'],df_temp['My_d'],c=df_temp['landing_rate'],cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing Rate')

ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)

ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')

ax.set_title('Raw Policy Relation (LR > 80%)')


plt.show()

# Note: the planar look to the data

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
## CLEAN DATAFRAME
df_temp = df.query(f"My_d<={7.7}") # Remove extraneous moments higher than max
df_temp = df_temp.query(f"RREV_threshold>{3.5}") # Remove outlier data
df_temp = df_temp.query(f"landing_rate>={0.5}")

## PLOT DATA
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")

cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)

ax.scatter(df_temp['RREV_threshold'],df_temp['OF_y'],df_temp['My_d'],c=df_temp['landing_rate'],cmap=cmap,norm=norm)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing Rate')

ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)

ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')

ax.set_title('Raw Policy Relation (LR > 50%)')


plt.show()

# Note: the planar look to the data

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# LINEAR MODEL FIT

## Regression with y-based loss

In [20]:
## CLEAN DATAFRAME
df_clean = df.query(f"My_d<={7.7}") # Remove extraneous moments higher than max
df_clean = df_clean.query(f"RREV_threshold>{3.5}") # Remove outlier data
df_reg1 = df_clean.query(f"landing_rate>={0.6}")


X = df_reg1[['RREV_threshold','OF_y']]
Y = df_reg1['My_d']

reg = linear_model.LinearRegression(normalize=True)
reg.fit(X,Y)
intercept = reg.intercept_
coeffs = reg.coef_

print(f"Equation: My_d = {intercept:.2f} + {coeffs[0]:.2f}*RREV + {coeffs[1]:.2f}*OF_y \n")


## PLOT DATA
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")


My_reg = intercept + coeffs[0]*df_reg1['RREV_threshold'] + coeffs[1]*df_reg1['OF_y']
ax.plot_trisurf(df_reg1['RREV_threshold'],df_reg1['OF_y'],My_reg,label='Linear_Regression',color='blue',zorder=2)


cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing Rate')

ax.scatter(df_reg1['RREV_threshold'],df_reg1['OF_y'],df_reg1['My_d'],c=df_reg1['landing_rate'],cmap=cmap,norm=norm,zorder=1)



ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)


ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')


plt.show()

## Regression Analysis

import statsmodels.api as ssm       # for detail description of linear coefficients, intercepts, deviations, and many more
X=ssm.add_constant(X)               # to add constant value in the model
model= ssm.OLS(Y,X).fit()           # fitting the model
predictions= model.summary()        # summary of the model
predictions





Equation: My_d = -3.61 + 1.79*RREV + 0.11*OF_y 



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   My_d   R-squared:                       0.134
Model:                            OLS   Adj. R-squared:                  0.128
Method:                 Least Squares   F-statistic:                     23.46
Date:                Tue, 09 Feb 2021   Prob (F-statistic):           3.36e-10
Time:                        19:51:17   Log-Likelihood:                -558.55
No. Observations:                 307   AIC:                             1123.
Df Residuals:                     304   BIC:                             1134.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -3.6073      1.291     -2.795      0.006      -6.147      -1.067
RREV_threshold     1.7898      0.312      5.745      0.000       1.177       2.403
OF_y               0.1090      0.071      1.537      0.125      -0.031       0.249
==============================================================================
Omnibus:                       23.867   Durbin-Watson:                   1.490
Prob(Omnibus):                  0.000   Jarque-Bera (JB):                8.915
Skew:                           0.109   Prob(JB):                       0.0116
Kurtosis:                       2.194   Cond. No.                         91.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [21]:
# def r_squared(y, y_hat): #Manual R^2 value
#     y_bar = y.mean()
#     ss_tot = ((y-y_bar)**2).sum()
#     ss_res = ((y-y_hat)**2).sum()
#     return 1 - (ss_res/ss_tot)

# R2 = r_squared(df_reg['My_d'],My_reg)
# print(f"R^2: {R2:.3f}")

## Regression with x-based loss


In [22]:
## CLEAN DATAFRAME
df_clean = df.query(f"My_d<={7.7}") # Remove extraneous moments higher than max
df_clean = df_clean.query(f"RREV_threshold>{3.5}") # Remove outlier data
df_reg1 = df_clean.query(f"landing_rate>={0.6}")


X = df_reg1[['My_d','OF_y']]
Y = df_reg1['RREV_threshold']

reg = linear_model.LinearRegression(normalize=True)
reg.fit(X,Y)
intercept = reg.intercept_
coeffs = reg.coef_

print(f"Equation: RREV_reg = {intercept:.2f} + {coeffs[0]:.2f}*My_d + {coeffs[1]:.2f}*OF_y \n")


## PLOT DATA
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")


RREV_reg = intercept + coeffs[0]*df_reg1['My_d'] + coeffs[1]*df_reg1['OF_y']
ax.plot_trisurf(RREV_reg,df_reg1['OF_y'],df_reg1['My_d'],label='Linear_Regression',color='blue',zorder=2)


cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing Rate')

ax.scatter(df_reg1['RREV_threshold'],df_reg1['OF_y'],df_reg1['My_d'],c=df_reg1['landing_rate'],cmap=cmap,norm=norm,zorder=1)



ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)


ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')


plt.show()

## Regression Analysis

import statsmodels.api as ssm       # for detail description of linear coefficients, intercepts, deviations, and many more
X=ssm.add_constant(X)               # to add constant value in the model
model= ssm.OLS(Y,X).fit()           # fitting the model
predictions= model.summary()        # summary of the model
predictions





Equation: RREV_reg = 3.90 + 0.05*My_d + -0.15*OF_y 



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         RREV_threshold   R-squared:                       0.564
Model:                            OLS   Adj. R-squared:                  0.561
Method:                 Least Squares   F-statistic:                     196.6
Date:                Tue, 09 Feb 2021   Prob (F-statistic):           1.61e-55
Time:                        19:51:19   Log-Likelihood:                -23.211
No. Observations:                 307   AIC:                             52.42
Df Residuals:                     304   BIC:                             63.60
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.8975      0.048     81.621      0.000       3.804       3.991
My_d           0.0547      0.010      5.745      0.000       0.036       0.073
OF_y          -0.1536      0.009    -17.455      0.000      -0.171      -0.136
==============================================================================
Omnibus:                        6.437   Durbin-Watson:                   1.269
Prob(Omnibus):                  0.040   Jarque-Bera (JB):                9.898
Skew:                          -0.008   Prob(JB):                      0.00709
Kurtosis:                       3.879   Cond. No.                         18.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [23]:
## PLOT FULL DATA
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")


ax.plot_trisurf(RREV_reg,df_reg1['OF_y'],df_reg1['My_d'],label='Linear_Regression',color='blue',zorder=2)


cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing Rate')

ax.scatter(df_clean['RREV_threshold'],df_clean['OF_y'],df_clean['My_d'],c=df_clean['landing_rate'],cmap=cmap,norm=norm,zorder=1)



ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)


ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')


plt.show()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
## CLEAN DATAFRAME
df_clean = df.query(f"My_d<={7.7}") # Remove extraneous moments higher than max
df_clean = df_clean.query(f"RREV_threshold>{3.5}") # Remove outlier data
df_reg1 = df_clean.query(f"landing_rate>={0.8}")


fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")


for ii in np.arange(1,0.2,-.2):
    df_reg1 = df.query(f"landing_rate<={ii} & landing_rate>{ii-.2} & RREV_threshold>{3.5}")
    
    X = df_reg1[['My_d','OF_y']]
    Y = df_reg1['RREV_threshold']

    reg2 = linear_model.LinearRegression(normalize=True)
    reg2.fit(X,Y)

    intercept = reg2.intercept_
    coeffs = reg2.coef_

    # print(intercept)
    # print(coeffs)

    RREV_reg = intercept + coeffs[0]*df_reg1['My_d'] + coeffs[1]*df_reg1['OF_y']
    ax.plot_trisurf(RREV_reg,df_reg1['OF_y'],df_reg1['My_d'],label='Linear_Regression')

    R2 = r_squared(df_reg1['RREV_threshold'],RREV_reg)
    print(f"R^2: {R2:.3f}")

ax.set_xlim(6,3.5)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)


ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')


plt.show()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'r_squared' is not defined

array([1. , 0.8, 0.6, 0.4])

In [ ]:
import scipy.odr

## CLEAN DATAFRAME
df_clean = df.query(f"My_d<={7.7}") # Remove extraneous moments higher than max
df_clean = df_clean.query(f"RREV_threshold>{3.5}") # Remove outlier data
df_reg1 = df_clean.query(f"landing_rate>={0.8}")


# X = df_reg1[['My_d','OF_y']]
x1 = df_reg1['RREV_threshold'].to_numpy()
x2 = df_reg1['OF_y'].to_numpy()
X = np.row_stack( (x1, x2) )

Y = df_reg1['My_d'].to_numpy()

def linfit(beta,x):
    return beta[0]*x[0] + beta[1]*x[1] + beta[2]

linmod = scipy.odr.Model(linfit)
data = scipy.odr.Data(X,Y)
odrfit = scipy.odr.ODR(data,linmod,beta0 = [-0.1,0.1,4.0])
odrres = odrfit.run()
odrres.pprint()




## PLOT DATA
fig = plt.figure()
ax = fig.add_subplot(111,projection="3d")


My_reg = odrres.beta[0]*df_reg1['RREV_threshold'] + odrres.beta[1]*df_reg1['OF_y'] + odrres.beta[2]
ax.plot_trisurf(df_reg1['RREV_threshold'],df_reg1['OF_y'],My_reg,label='Linear_Regression',color='blue',zorder=2)


cmap = mpl.cm.rainbow
norm = mpl.colors.Normalize(vmin=0,vmax=1)
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),label='Landing Rate')

ax.scatter(df_reg1['RREV_threshold'],df_reg1['OF_y'],df_reg1['My_d'],c=df_reg1['landing_rate'],cmap=cmap,norm=norm,zorder=1)



ax.set_xlim(6,2)
ax.set_ylim(-10,0)
ax.set_zlim(0,10)


ax.set_xlabel('RREV')
ax.set_ylabel('OF_y')
ax.set_zlabel('My_d')


plt.show()
